In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import data
from gensim.models import word2vec
from tqdm import tqdm
tqdm.pandas()

np.random.seed(42)

In [3]:
# This is the list of all the appellate cases from the 9th circuit

df=pd.read_pickle('./circuit_w_text')

In [5]:
df.loc[200]

,absolute_url,attorney,author_id,caseName,caseNameShort,citation,citeCount,cites,cluster_id,court,...,scdb_id,sibling_ids,snippet,source,status,status_exact,suitNature,timestamp,type,text
0,/opinion/451608/friends-of-endangered-species-...,,None,"Friends of Endangered Species, Inc. v. Robert ...",,[760 F.2d 976],124,"[109827, 110953, 313662, 329593, 374746, 38740...",451608,Court of Appeals for the Ninth Circuit,...,,[451608],"that, pursuant to <mark>section</mark> <mark>7...",R,Precedential,Precedential,,2017-08-01T17:27:51.200000Z,010combined,"760 F.2d 976\n22 ERC 1817, 15 Envtl. L. Rep. 2..."
1,/opinion/576710/mt-graham-red-squirrel-tamiasc...,,None,"Mt. Graham Red Squirrel, (Tamiasciurus Hudsoni...",,[954 F.2d 1441],102,"[107036, 107414, 108502, 108782, 109117, 11021...",576710,Court of Appeals for the Ninth Circuit,...,,[576710],86\n [T]he supporters of the <mark>...,R,Precedential,Precedential,,2018-04-23T17:27:43.386000Z,010combined,"954 F.2d 1441\n21 Fed.R.Serv.3d 1301, 22 Envtl..."
2,/opinion/601475/mt-graham-red-squirrel-tamiasc...,,None,"Mt. Graham Red Squirrel, (Tamiasciurus Hudsoni...",,[986 F.2d 1568],65,"[110400, 111373, 111789, 112256, 112747, 38008...",601475,Court of Appeals for the Ninth Circuit,...,,[601475],observatory project based on <mark>section</ma...,R,Precedential,Precedential,,2017-04-17T21:09:59.002000Z,010combined,"986 F.2d 1568\n81 Ed. Law Rep. 439, 23 Envtl. ..."
3,/opinion/661521/greenpeace-action-a-non-profit...,,None,"Greenpeace Action, a Non-Profit Corporation v....",,[14 F.3d 1324],148,"[108283, 109518, 111456, 112256, 313342, 40595...",661521,Court of Appeals for the Ninth Circuit,...,,[661521],"-4370d, and <mark>section</mark> <mark>7</mark...",R,Precedential,Precedential,,2017-12-27T18:55:43.045999Z,010combined,"14 F.3d 1324\nGREENPEACE ACTION,* a non-profit..."
4,/opinion/447020/harold-thomas-dba-allison-ranc...,,None,"Harold Thomas, Dba Allison Ranch and Cook Ranc...",,[753 F.2d 754],160,"[106984, 109518, 109897, 110102, 110112, 29955...",447020,Court of Appeals for the Ninth Circuit,...,,[447020],concerned the <mark>ESA</mark>&#39;s prohibiti...,R,Precedential,Precedential,,2018-03-19T22:47:26.045000Z,010combined,"753 F.2d 754\n22 ERC 1608, 15 Envtl. L. Rep. 2..."


In [35]:
# This the list of all the appellate cases except the 9th circuit

df2=pd.read_pickle('./circuit_dc_w_text')

In [36]:
df.shape

(264, 40)

In [37]:
df2.shape

(208, 40)

In [38]:
# Drop out entries that are blank or NAN in the text field.

df=df.dropna(subset=['text'])

df=df.loc[df['text']!='']

df2=df2.dropna(subset=['text'])

df2=df2.loc[df2['text']!='']

In [39]:
df.shape

(247, 40)

In [40]:
df2.shape

(205, 40)

In [41]:
# Adapted from Matt Brems' lecture notes

def review_to_wordlist(sentence):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #    
    
    # tokenizing and removing punctuation
    tokenizer = RegexpTokenizer(r'[a-zA-Z]{3,}')
    text_processed=tokenizer.tokenize(sentence)
    
    legal_words=set(['sec', 'fed', 'reg', 'act', 'cir', 'cert', 'see', 'app', 'soc', 'stat'])
    stops=set(stopwords.words('english'))
    
    # removing any stopwords
    text_processed = [word for word in text_processed  if (word.lower() not in stops and 
                     word.lower() not in legal_words)]
          
    # make words lower-cased, unless it is an acronym
    text_processed = [word.lower() if (word.upper() != word or len(word)>4) else word for word in text_processed ]
    
    # return a list of words
    return (text_processed)
    

In [42]:
# Adapted from Matt Brems' lecture notes.

# Load the punkt tokenizer
tokenizer = data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( text, tokenizer):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the document into sentences
    raw_sentences = tokenizer.tokenize(text.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [44]:
#Break each case down into a series of sentances, each of which is a list of words
# Assemble a list of all such sentences in the entire corpus


train_sentences = []  # Initialize an empty list of sentences
counter=0

print("Parsing sentences from training set")
for case_text in df['text']:
    print (counter)
    train_sentences += review_to_sentences(case_text, tokenizer)
    counter += 1

Parsing sentences from training set
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246


In [45]:
len(train_sentences)

109863

In [46]:
# Build a Word2Vec Model from the texts of all the cases


# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 100    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 40          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)

print("Training model...")
model = word2vec.Word2Vec(train_sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "100features_40minwords_40context"
model.save(model_name)

2018-07-06 21:33:11,872 : INFO : collecting all words and their counts
2018-07-06 21:33:11,875 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-07-06 21:33:11,917 : INFO : PROGRESS: at sentence #10000, processed 98026 words, keeping 7796 word types
2018-07-06 21:33:11,954 : INFO : PROGRESS: at sentence #20000, processed 194672 words, keeping 11582 word types
2018-07-06 21:33:11,990 : INFO : PROGRESS: at sentence #30000, processed 293803 words, keeping 13854 word types
2018-07-06 21:33:12,020 : INFO : PROGRESS: at sentence #40000, processed 395471 words, keeping 16096 word types
2018-07-06 21:33:12,061 : INFO : PROGRESS: at sentence #50000, processed 497056 words, keeping 17779 word types


Training model...


2018-07-06 21:33:12,097 : INFO : PROGRESS: at sentence #60000, processed 600534 words, keeping 18974 word types
2018-07-06 21:33:12,131 : INFO : PROGRESS: at sentence #70000, processed 699081 words, keeping 19963 word types
2018-07-06 21:33:12,158 : INFO : PROGRESS: at sentence #80000, processed 773462 words, keeping 20600 word types
2018-07-06 21:33:12,195 : INFO : PROGRESS: at sentence #90000, processed 879668 words, keeping 22230 word types
2018-07-06 21:33:12,231 : INFO : PROGRESS: at sentence #100000, processed 980906 words, keeping 23205 word types
2018-07-06 21:33:12,267 : INFO : collected 24830 word types from a corpus of 1080367 raw words and 109863 sentences
2018-07-06 21:33:12,268 : INFO : Loading a fresh vocabulary
2018-07-06 21:33:12,294 : INFO : min_count=40 retains 3619 unique words (14% of original 24830, drops 21211)
2018-07-06 21:33:12,295 : INFO : min_count=40 leaves 948364 word corpus (87% of original 1080367, drops 132003)
2018-07-06 21:33:12,317 : INFO : deleting 

In [47]:
# Process each text field into a list of tokens

df['text2']=df['text'].progress_apply(review_to_wordlist)

100%|██████████| 247/247 [00:01<00:00, 134.78it/s]


In [48]:
# Process each text field into a list of tokens

df2['text2']=df2['text'].progress_apply(review_to_wordlist)

100%|██████████| 205/205 [00:01<00:00, 137.35it/s]


In [51]:
df.to_pickle('./processed_text')

In [52]:
df2.to_pickle('./dc_processed_text')

In [53]:
model.wv.vocab

{'envtl': <gensim.models.keyedvectors.Vocab at 0x1a1ec3b550>,
 'rep': <gensim.models.keyedvectors.Vocab at 0x1a1d6fa7b8>,
 'friends': <gensim.models.keyedvectors.Vocab at 0x1a21026e48>,
 'endangered': <gensim.models.keyedvectors.Vocab at 0x1a21026940>,
 'species': <gensim.models.keyedvectors.Vocab at 0x1a21026400>,
 'INC': <gensim.models.keyedvectors.Vocab at 0x1a210267f0>,
 'plaintiff': <gensim.models.keyedvectors.Vocab at 0x1a21026c18>,
 'appellant': <gensim.models.keyedvectors.Vocab at 0x1a210269e8>,
 'robert': <gensim.models.keyedvectors.Vocab at 0x1a21026dd8>,
 'director': <gensim.models.keyedvectors.Vocab at 0x1a21026668>,
 'united': <gensim.models.keyedvectors.Vocab at 0x1a21026470>,
 'states': <gensim.models.keyedvectors.Vocab at 0x1a210265f8>,
 'fish': <gensim.models.keyedvectors.Vocab at 0x1a21026b38>,
 'county': <gensim.models.keyedvectors.Vocab at 0x1a21026780>,
 'san': <gensim.models.keyedvectors.Vocab at 0x1a21026128>,
 'city': <gensim.models.keyedvectors.Vocab at 0x1a210

In [59]:
model.most_similar('pesticide')

/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('reregistration', 0.8935496211051941),
 ('EPA', 0.8062836527824402),
 ('RED', 0.7879930734634399),
 ('registration', 0.6772563457489014),
 ('fifra', 0.6744117140769958),
 ('pesticides', 0.6596125364303589),
 ('sub', 0.5840566754341125),
 ('category', 0.5825705528259277),
 ('active', 0.5770189762115479),
 ('npdes', 0.5675276517868042)]